In [197]:
import sqlite3
import sys
sys.path.append('../../')
from src.preprocessor.preprocessing import FileIO
from src.llm.llm_interface import LLM
from src.llm.llm_utils import load_azure_openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(), override=True)

In [198]:
llm = load_azure_openai()

In [195]:
file = 'sqlite3.db'
data = FileIO.load_json('../../data/huberman_labs.json')

In [208]:
data_dir = '../../data/'
data = []
names = ['guests', 'titles', 'summaries', 'view_counts']
for name in names:
    data.append(tuple(FileIO.load_json(f'{data_dir}{name}.json')))

In [216]:
guests, titles, summaries, view_counts = [*data]

Bad pipe message: %s [b'(>kz\xcbr\xe7\xf7[\x13\xe3\\M\x9a}4T\x15 \x12_87\xd6b\xbd\xd1t\x8e\xa5\x1a\x87\xbdn\x9c38\xbb\xc6\x8e\x8a\x16\x9e\\;\x1a\xe31\x151\xae\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00', b'\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05']
Bad pipe message: %s [b'\x03\x08']
Bad pipe message: %s [b'\x08\x08\t\x08\n\x08']
Bad pipe message: %s [b'\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06']
Bad pipe message: %s [b'']
Bad pipe message: %s [b"\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 \xbd\xe5\xa2\x18B^x '\xce\x87U\x10\xa6\xaa\x06a@N6HY"]
Bad pipe message: %s [b"\xf0\xa8\xf5W@\x1duq0\xc98F\xcb@\xbb\xabk?\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W\xc0S\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\

In [3]:
conn= sqlite3.connect(file)

In [5]:
cursor = conn.cursor()

In [6]:
table_schema = """
CREATE TABLE IF NOT EXISTS huberman (
    id INTEGER PRIMARY KEY,
    guests TEXT NOT NULL,
    titles TEXT NOT NULL,
    summaries TEXT NOT NULL,
    view_counts INTEGER NOT NULL
);
"""

In [7]:
cursor.execute(table_schema)

In [8]:
cursor.execute('SELECT * from huberman;')

In [9]:
result = cursor.fetchall()

In [10]:
result

[]

In [32]:
def insert_rows(guests, titles, summaries, counts): 
    connection = sqlite3.connect(file)
    cursor = connection.cursor()
    for guest, title, summary, count in zip(guests, titles, summaries, counts):
        insert_query =  f"""
                        INSERT INTO huberman (guests, titles, summaries, view_counts)
                        VALUES (?,?,?,?);
                        """
        cursor.execute(insert_query, (guest, title, summary, count))
    connection.commit()
    cursor.close()
    connection.close()

In [33]:
insert_rows(guests, titles, summaries, counts)

In [77]:
def get_cursor():
    conn = sqlite3.connect(file)
    cursor = conn.cursor()
    return cursor, conn

In [171]:
def get_response(command: str):
    cursor, conn = get_cursor()
    cursor.execute(command)
    answer = cursor.fetchall()
    cursor.close()
    conn.close()
    return answer

In [73]:
c = get_cursor()

In [36]:
c.execute('SELECT * FROM huberman;')

In [37]:
result = c.fetchall()

In [46]:
system_message = """
Your main function is to generate flawless SQL queries from plain text language, that will enable a user 
to query a SQLite3 database.  There is one table called huberman in the database with four columns labeled:
1. guests
2. titles
3. summaries
4. view_counts
"""

In [65]:
base_user_message = """
Convert the plain text query into a SQL query that can answer questions about the data contained in the huberman table. 
Output your response in JSON format, do not include any newlines, and return only the SQL command nothing else.
--------------
QUERY: {query}
--------------
SQL:
"""

In [94]:
response_system_message = """
You excel at answering questions from a multitude of information sources.  Specifically you know how to translate SQL-generated answers into actionable insights
"""

In [174]:
base_response_user_message = """
You will be given a user question and a response to that query pulled from a SQL database.  
Use the information from the response to answer the original query.  Answer the question in an objective tone, and do not make any reference to the SQL database.
If the information in the answer is presented in a list, ensure that you return your output in the same order, do not mix up the original ordering. 
If a specific piece of text information is requested such as a guest name or show title, use the LIKE operator and % wildcard character as appropriate to address the user information need.
If the response does not answer the question, then either state that you do not know the answer, or answer the question based on prior knowledge.
----------------
USER QUERY: {query}
----------------
ANSWER FROM SQL DATABASE: {sql_answer}
----------------
OUTPUT: 
"""

In [183]:
query = 'Return show titles that discuss muscle hypertrophy with over 500K views'
user_message = base_user_message.format(query=query)
response = llm.chat_completion(system_message, user_message)
answer = get_response(response)
post_user_message = base_response_user_message.format(query=query, sql_answer=answer)
post_response = llm.chat_completion(response_system_message, post_user_message)

In [184]:
print(response)
print(answer)
print(post_response)

SELECT titles 
FROM huberman 
WHERE summaries LIKE '%muscle hypertrophy%' 
AND view_counts > 500000;
[('Dr. Andy Galpin: How to Assess & Improve All Aspects of Your Fitness | Huberman Lab Guest Series',), ('Science of Muscle Growth, Increasing Strength & Muscular Recovery',)]
The show titles that discuss muscle hypertrophy with over 500K views are:
1. Dr. Andy Galpin: How to Assess & Improve All Aspects of Your Fitness | Huberman Lab Guest Series
2. Science of Muscle Growth, Increasing Strength & Muscular Recovery


In [129]:
print(response)

SELECT titles 
FROM huberman 
WHERE summaries LIKE '%mental health%' 
AND view_counts > 1000000


In [130]:
conn=sqlite3.connect(file)
cur = conn.cursor()

In [158]:
cur.execute("SELECT titles FROM huberman WHERE summaries LIKE '%mental health%' AND view_counts > 100000")

In [159]:
new_answer = cur.fetchall()
new_answer

[('LIVE EVENT Q&A: Dr. Andrew Huberman Question & Answer in Melbourne, AU',),
 ('How Placebo Effects Work to Change Our Biology & Psychology',),
 ('How to Improve Oral Health & Its Critical Role in Brain & Body Health',),
 ('Dr. Kay Tye: The Biology of Social Interactions and Emotions',),
 ('Journal Club with Dr. Peter Attia | Effects of Light & Dark on Mental Health & Treatments for Cancer',),
 ('LIVE EVENT Q&A: Dr. Andrew Huberman Question & Answer in Toronto, ON',),
 ('AMA #12: Thoughts on Longevity Supplements (Resveratrol, NR, NMN, Etc.) & How to Improve Memory',),
 ('Mental Health Toolkit: Tools to Bolster Your Mood & Mental Health',),
 ('Mark Zuckerberg & Dr. Priscilla Chan: Curing All Human Diseases & the Future of Health & Technology',),
 ('Dr. Paul Conti: Tools and Protocols for Mental Health | Huberman Lab Guest Series',),
 ('Dr. Paul Conti: How to Improve Your Mental Health | Huberman Lab Guest Series',),
 ('Dr. Paul Conti: How to Understand & Assess Your Mental Health | Hu

In [161]:
post_user_message = base_response_user_message.format(query=query, sql_answer=new_answer)
post_response = llm.chat_completion(response_system_message, post_user_message, max_tokens=1500)
print(post_response)

Based on the information provided, the show titles that discussed mental health and had over 100K views are as follows:

1. LIVE EVENT Q&A: Dr. Andrew Huberman Question & Answer in Melbourne, AU
2. How Placebo Effects Work to Change Our Biology & Psychology
3. How to Improve Oral Health & Its Critical Role in Brain & Body Health
4. Dr. Kay Tye: The Biology of Social Interactions and Emotions
5. Journal Club with Dr. Peter Attia | Effects of Light & Dark on Mental Health & Treatments for Cancer
6. LIVE EVENT Q&A: Dr. Andrew Huberman Question & Answer in Toronto, ON
7. AMA #12: Thoughts on Longevity Supplements (Resveratrol, NR, NMN, Etc.) & How to Improve Memory
8. Mental Health Toolkit: Tools to Bolster Your Mood & Mental Health
9. Mark Zuckerberg & Dr. Priscilla Chan: Curing All Human Diseases & the Future of Health & Technology
10. Dr. Paul Conti: Tools and Protocols for Mental Health | Huberman Lab Guest Series
11. Dr. Paul Conti: How to Improve Your Mental Health | Huberman Lab Gue